## Практические задания

**Задание 2.**

Необходимо собрать информацию по продуктам питания с сайтов:
Роскачество официальный сайт. Исследование качества продуктов питания | Рейтинг товаров.
Список протестированных продуктов на сайте Росконтроль.рф
Получившийся список должен содержать:
Наименование продукта.
Категорию продукта (например «Бакалея»).
Подкатегорию продукта (например «Рис круглозерный»).
Параметр «Безопасность».
Параметр «Качество».
Общий балл.
Сайт, откуда получена информация. ### Структура должна быть одинаковая для продуктов с обоих сайтов. Общий результат можно вывести с помощью dataFrame через Pandas.

**Подключение библиотек и скриптов**

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import re

**Вспомогательные переменные**

In [2]:
URL_RSK = 'https://rskrf.ru'
URL_RK = 'https://roscontrol.com/category/produkti'
URL_RK_s = 'https://roscontrol.com'
N_PAGES = 3

**Вспомогательные процедуры и функции**

In [11]:
def get_products_RSK():
    
    GET_PRODUCTS_REQUEST = URL_RSK + '/ratings/produkty-pitaniya/'
    HEADER = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
    
    data_list = []
    
    data = requests.get(GET_PRODUCTS_REQUEST, headers = HEADER)
    html = BeautifulSoup(data.text, 'lxml')
    tags = html.find_all('div', attrs = {'class': 'category-item'})
    
    a = 0
    
    for tag in tags:
        
        a += 1
        
        if a > 5: break
        
        #ссылки на категории
        href_name = tag.find('a')
        cat_path = URL_RSK + href_name['href']
        data_sub = requests.get(cat_path, headers = HEADER)
        html_sub = BeautifulSoup(data_sub.text, 'lxml')
        tags_sub = html_sub.find_all('div', attrs = {'class': 'category-item'})
        name_path = html_sub.find('li', attrs = {'class': 'breadcrumb-item active'}).text
        
        i = 0
        
        for tag_sub in tags_sub:
            
            i += 1
            
            if i > 5: break
            
            subcat_path = tag_sub.find('a')
            cat_path_sub = URL_RSK + subcat_path['href']
            name_subpath = subcat_path.text
            
            data_cards = requests.get(cat_path_sub, headers = HEADER)
            html_cards = BeautifulSoup(data_cards.text, 'lxml')
            name_subpath = html_cards.find('li', attrs = {'class': 'breadcrumb-item active'}).text

            tags_cards__ = html_cards.find_all('noscript')
            
            j = 0
            
            for tag_card__ in tags_cards__:
                 
                j += 1
                
                if j > 10: break
                
                a_tags = tag_card__.find_all('a')
                
                if len(a_tags) > 0:
                    hrefs_card = tag_card__.find_all('a')
                    
                    k = 0
                    
                    for href_card in hrefs_card:
                        
                        k += 1
                        
                        if k > 5: break
                        
                        name = href_card.text
                        final_card = requests.get(URL_RSK + href_card['href'], headers = HEADER)
                        html_card = BeautifulSoup(final_card.text, 'lxml')

                        total_rate_html = html_card.find('div', attrs = {'class': 'col-12 d-none d-md-block order-0'})
                        total_rate = total_rate_html.find_all('span')[1].text
                        html_rates = html_card.find('div', attrs = {'class': 'col-12 product-rating'}).find_all('div', attrs = {'class': 'rating-item'})
                        
                        safety = ''
                        quality = ''
                        
                        for rate in html_rates:
                            
                            spans = rate.find_all('span')
                            
                            if spans[0].text.strip() == 'Безопасность': 
                                safety = spans[1].text.strip()
                                
                            if spans[0].text.strip() == 'Качество': 
                                quality = spans[1].text.strip()
                                

                        data_list.append(['https://rskrf.ru', name, name_path, name_subpath, safety, quality, total_rate])
                        
    df = pd.DataFrame(data_list, columns = ['Resource', 'Name', 'Categoty', 'Sub category', 'Safety Rate', 'Quality Rate', 'Total Rate'])
        
    return df

In [12]:
def get_products_RK():
    
    GET_PRODUCTS_REQUEST = URL_RK + '/#'
    HEADER = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
    
    data_list = []
    
    data = requests.get(GET_PRODUCTS_REQUEST, headers = HEADER)
    html = BeautifulSoup(data.text, 'lxml')
    tags = html.find_all('div', attrs = {'class': 'grid-padding grid-column-3 grid-column-large-6 grid-flex-mobile grid-column-middle-6 grid-column-small-12 grid-left'})
    
    a = 0
    
    for tag in tags:
        
        a += 1
        
        if a > 5: break
        
        #ссылки на категории
        href_name = tag.find('a')
        name_path = tag.find('div', attrs = {'class': 'catalog__category-name'}).text
        cat_path = URL_RK + href_name['href']

        data_sub = requests.get(cat_path, headers = HEADER)
        html_sub = BeautifulSoup(data_sub.text, 'lxml')
        tags_sub = html_sub.find_all('div', attrs = {'class': 'grid-padding grid-column-3 grid-column-large-6 grid-flex-mobile grid-column-middle-6 grid-column-small-12 grid-left'})
                
        i = 0
        
        for tag_sub in tags_sub:
            
            i += 1
            
            if i > 10: break
            
            subcat_path = tag_sub.find('a')
            cat_path_sub = URL_RK + subcat_path['href']
            name_subpath = tag_sub.find('div', attrs = {'class': 'catalog__category-name'}).text
            
            data_cards = requests.get(cat_path_sub, headers = HEADER)
            html_cards = BeautifulSoup(data_cards.text, 'lxml')

            tags_cards__ = html_cards.find_all('div', attrs = {'class': 'wrap-product-catalog__item grid-padding grid-column-4 grid-column-large-6 grid-column-middle-12 grid-column-small-12 grid-left js-product__item'})
            
            j = 0
            
            for tag_card__ in tags_cards__:
                
                j += 1
                
                if j > 5: break
                
                href_card = tag_card__.find('a')
                name = tag_card__.find('div', attrs = {'class': 'product__item-link'}).text
                
                final_card = requests.get(URL_RK_s + href_card['href'], headers = HEADER)
                html_card = BeautifulSoup(final_card.text, 'lxml')
                
                total_rate_ = html_card.find('div', attrs = {'class': 'product__single-rev-total'})
                
                total_rate = ''
                
                if total_rate_:
                    total_rate = total_rate_.find('div').text
                
                html_rates = html_card.find_all('div', attrs = {'class': 'rate-item group'})
                
                safety = ''
                quality = ''
                
                for rate in html_rates:
        
                    span_text = rate.find('div', attrs = {'class': 'rate-item__title'}).text.strip()
                    
                    if span_text == 'Безопасность': 
                        safety = rate.find('span').text
                    if span_text == 'Качество': 
                        quality = rate.find('span').text


                    data_list.append(['https://roscontrol.com', name, name_path, name_subpath, safety, quality, total_rate])
                        
    df = pd.DataFrame(data_list, columns = ['Resource', 'Name', 'Categoty', 'Sub category', 'Safety Rate', 'Quality Rate', 'Total Rate'])
        
    return df

In [ ]:
df = pd.concat([get_products_RSK(), get_products_RK()])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 758 entries, 0 to 652
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Resource      758 non-null    object
 1   Name          758 non-null    object
 2   Categoty      758 non-null    object
 3   Sub category  758 non-null    object
 4   Safety Rate   758 non-null    object
 5   Quality Rate  758 non-null    object
 6   Total Rate    758 non-null    object
dtypes: object(7)
memory usage: 47.4+ KB


In [10]:
df

,Resource,Name,Categoty,Sub category,Safety Rate,Quality Rate,Total Rate
0,https://rskrf.ru,Barilla - Макаронные изделия из твердых сортов...,Бакалея,Макаронные изделия «перья» из твердых сортов п...,5.000,5.000,[5.0000]
1,https://rskrf.ru,La Molisana - Изделия макаронные из твердых со...,Бакалея,Макаронные изделия «перья» из твердых сортов п...,5.000,5.000,[5.0000]
2,https://rskrf.ru,Итальянские - Изделия макаронные из пшеничной ...,Бакалея,Макаронные изделия «перья» из твердых сортов п...,5.000,5.000,[5.0000]
3,https://rskrf.ru,"Роллтон - Изделия макаронные «перья», группа А...",Бакалея,Макаронные изделия «перья» из твердых сортов п...,5.000,5.000,[5.0000]
4,https://rskrf.ru,"Grand di Pasta - Изделия макаронные, группа А,...",Бакалея,Макаронные изделия «перья» из твердых сортов п...,5.000,4.730,[4.8650]
...,...,...,...,...,...,...,...
648,https://roscontrol.com,"""О`кей / Родной бюветъ №17, Ессентуки-город к...","Вода, соки, напитки",Вода лечебная минеральная,,,64
649,https://roscontrol.com,"""О`кей / Родной бюветъ №17, Ессентуки-город к...","Вода, соки, напитки",Вода лечебная минеральная,49,,64
650,https://roscontrol.com,"""Старый источникъ №17""","Вода, соки, напитки",Вода лечебная минеральная,,,63
651,https://roscontrol.com,"""Старый источникъ №17""","Вода, соки, напитки",Вода лечебная минеральная,,,63
